In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

fname='./brazil_cities.csv'

df=pd.read_csv(fname)

cities={}

for index, row in df.iterrows():
    city_name=row['URL'].split('/')[-1]
    
    cities[city_name]=[
        float(row['latitude'].replace(',', '.')), 
        float(row['longitude'].replace(',', '.'))
    ]

In [7]:
import urllib.parse

cities = {urllib.parse.unquote(city_name): coordinates for city_name, coordinates in cities.items()}


In [10]:
from spycio import distance
from tqdm import tqdm

k=10
radius=6371
config={'radius':radius}

dist_fun=lambda A, B: distance(A, B, 'geographical', config)

from multiprocessing import Process

# Create a new process
process = Process()

closest_cities={}

for city_name in tqdm(cities.keys()):
    city_position=cities[city_name]
    
    dist_tuple_fun=lambda candidate: (candidate[0], dist_fun(city_position, candidate[1]))
    dist_lst=list(
        map(
            dist_tuple_fun, 
            [city_item for city_item in list(cities.items()) if city_item[0] != city_name]
        )
    )
    
    dist_lst.sort(key=lambda x: x[1])
    
    closest_cities[city_name]=dist_lst[:k]

  9%|███▌                                    | 493/5554 [01:33<16:18,  5.17it/s]/home/brunolnetto/.local/lib/python3.8/site-packages/spycio/spycio.py:85: RuntimeWarning: invalid value encountered in arccos
  return arccos(cosnuv(u, v, n))
100%|███████████████████████████████████████| 5554/5554 [18:41<00:00,  4.95it/s]


In [11]:
dist_fun(cities['Itauçu'], cities['Inhumas'])

20.46451941262333

In [12]:
import json

# Serializing json
json_object = json.dumps(closest_cities, indent=4)

fname="{k}_closest_cities.json".format(k=k)

# Writing to sample.json
with open(fname, "w") as outfile:
    outfile.write(json_object)